# **W1D1: Introduction to the Climate System and Xarray**

## Overview

The first three tutorials of today will introduce the basics of gridded, labeled data with Xarray. Since Xarray introduces additional abstractions on top of plain arrays of data, our goal is to show why these abstractions are useful and how they frequently lead to simpler, more robust code.

We'll cover these topics:

1. Create a `DataArray`, one of the core object types in Xarray
1. Understand how to use named coordinates and metadata in a `DataArray`
1. Combine individual `DataArrays` into a `Dataset`, the other core object type in Xarray
1. Subset, slice, and interpolate the data using named coordinates
1. Open netCDF data using XArray
1. Basic subsetting and aggregation of a `Dataset`
1. Brief introduction to plotting with Xarray

# **Tutorial 1: Creating DataArrays and Datasets to Assess Global Climate Data**


**Week 1, Day 1, Introduction to the Climate System**

**Content creators:** Sloane Garelick, Julia Kent

**Content reviewers:** Danika Gupta, Younkap Nina Duplex 

**Content editors:** Yosmely Bermúdez

**Production editors:** TBD

**Our 2023 Sponsors:** TBD

###**Code and Data Sources**

Code and data for this tutorial is based on existing content from [Project Pythia](https://foundations.projectpythia.org/core/xarray/xarray-intro.html).

## **Tutorial 1 Objectives**


As you just learned in the Introduction to Climate video, variations in global climate involve various forcings and interactions between multiple systems. Because of this complexity, global climate datasets are often very large with multiple dimensions and variables.

One useful computational tool for organizing, analyzing and interpreting large global datasets is XArray, an open source project and Python package that makes working with labelled multi-dimensional arrays simple and efficient.

In this first tutorial, we will use the `DataArray` and `Dataset` objects to practice organizing large datasets.

## Imports

Simmilar to `numpy`, `np`; `pandas`, `pd`; you may often encounter `xarray` imported within a shortened namespace as `xr`. `pythia_datasets` provides example data for us to work with.

In [ ]:
!pip install datetime

!pip install numpy
!pip install pandas
!pip install xarray
!pip install pythia_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 4.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datetime import timedelta

import numpy as np
import pandas as pd
import xarray as xr
from pythia_datasets import DATASETS

## Introducing the `DataArray` and `Dataset`

Xarray expands on the capabilities on NumPy arrays, providing a lot of streamlined data manipulation. It is similar in that respect to Pandas, but whereas Pandas excels at working with tabular data, Xarray is focused on N-dimensional arrays of data (i.e. grids). Its interface is based largely on the netCDF data model (variables, attributes, and dimensions), but it goes beyond the traditional netCDF interfaces to provide functionality similar to netCDF-java's [Common Data Model (CDM)](https://docs.unidata.ucar.edu/netcdf-java/current/userguide/common_data_model_overview.html). 

### Creation of a `DataArray` object

The `DataArray` is one of the basic building blocks of Xarray (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray)). It provides a `numpy.ndarray`-like object that expands to provide two critical pieces of functionality:

1. Coordinate names and values are stored with the data, making slicing and indexing much more powerful
2. It has a built-in container for attributes

Here we'll initialize a `DataArray` object by wrapping a plain NumPy array, and explore a few of its properties.

#### Generate a random numpy array

For our first example, we'll just create a random array of "temperature" data in units of Kelvin:

In [ ]:
data = 283 + 5 * np.random.randn(5, 3, 4)
data

array([[[283.08541883, 288.66756526, 281.38769563, 289.09592436],
        [286.93618323, 280.95953503, 285.6377926 , 291.67822026],
        [280.02890318, 278.76181509, 279.00394902, 280.67051413]],

       [[283.83313225, 280.05227571, 278.07358383, 282.30719687],
        [288.11441849, 282.75544937, 282.17615764, 283.50768661],
        [286.94383421, 298.17221264, 282.53607365, 284.95647635]],

       [[271.44742019, 283.84540994, 288.65788172, 278.58877862],
        [282.52040833, 275.43533336, 285.45025005, 279.63224775],
        [279.90644577, 287.47469906, 276.94072522, 280.44190702]],

       [[273.54477757, 289.66800952, 272.08984156, 282.07947272],
        [284.17157693, 283.68730888, 287.04087551, 285.90846735],
        [281.93633873, 278.57805726, 283.12871719, 296.02882419]],

       [[295.15387515, 276.33878186, 281.76181383, 274.16344205],
        [286.34442618, 286.89832226, 283.97439779, 283.43792752],
        [284.88351424, 294.58047289, 276.95646721, 281.99286514]]])

#### Wrap the array: first attempt

Now we create a basic `DataArray` just by passing our plain `data` as input:

In [ ]:
temp = xr.DataArray(data)
temp

<xarray.DataArray (dim_0: 5, dim_1: 3, dim_2: 4)>
array([[[283.08541883, 288.66756526, 281.38769563, 289.09592436],
        [286.93618323, 280.95953503, 285.6377926 , 291.67822026],
        [280.02890318, 278.76181509, 279.00394902, 280.67051413]],

       [[283.83313225, 280.05227571, 278.07358383, 282.30719687],
        [288.11441849, 282.75544937, 282.17615764, 283.50768661],
        [286.94383421, 298.17221264, 282.53607365, 284.95647635]],

       [[271.44742019, 283.84540994, 288.65788172, 278.58877862],
        [282.52040833, 275.43533336, 285.45025005, 279.63224775],
        [279.90644577, 287.47469906, 276.94072522, 280.44190702]],

       [[273.54477757, 289.66800952, 272.08984156, 282.07947272],
        [284.17157693, 283.68730888, 287.04087551, 285.90846735],
        [281.93633873, 278.57805726, 283.12871719, 296.02882419]],

       [[295.15387515, 276.33878186, 281.76181383, 274.16344205],
        [286.34442618, 286.89832226, 283.97439779, 283.43792752],
        [284.88351424, 294.58047289, 276.95646721, 281.99286514]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

Note two things:

1. Xarray generates some basic dimension names for us (`dim_0`, `dim_1`, `dim_2`). We'll improve this with better names in the next example.
2. Wrapping the numpy array in a `DataArray` gives us a rich display in the notebook! (Try clicking the array symbol to expand or collapse the view)

#### Assign dimension names

Much of the power of Xarray comes from making use of named dimensions. So let's add some more useful names! We can do that by passing an ordered list of names using the keyword argument `dims`:

In [ ]:
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[276.91512138, 279.86387969, 278.36458624, 273.0550014 ],
        [288.20363005, 277.39943919, 276.14656492, 273.5040345 ],
        [277.05082755, 292.50133624, 283.92811403, 278.48443179]],

       [[283.58293294, 276.11952674, 288.31042283, 281.07499717],
        [285.37992311, 286.49367673, 280.24596734, 285.20312622],
        [287.23002777, 284.54913327, 290.31045538, 278.10829299]],

       [[276.47179621, 284.62000068, 279.92057533, 279.35633454],
        [286.23998898, 280.63052782, 287.89197253, 283.0267185 ],
        [287.91121394, 275.14375092, 292.06369705, 282.1072505 ]],

       [[280.91171542, 280.06454881, 294.56050896, 279.73469873],
        [280.71513053, 283.78874772, 280.85957937, 276.17142697],
        [273.80101824, 287.92413243, 291.89451817, 289.16031958]],

       [[283.49367266, 284.20561716, 284.96659708, 284.60470716],
        [280.27024343, 288.30916921, 279.4805367 , 285.53285659],
        [285.91854135, 277.46997501, 276.42230361, 282.76974697]]])
Dimensions without coordinates: time, lat, lon

This is already improved upon from a NumPy array, because we have names for each of the dimensions (or axes in NumPy parlance). Even better, we can take arrays representing the values for the coordinates for each of these dimensions and associate them with the data when we create the `DataArray`. We'll see this in the next example.

### Create a `DataArray` with named Coordinates

#### Make time and space coordinates

Here we will use [Pandas](../pandas) to create an array of [datetime data](../datetime), which we will then use to create a `DataArray` with a named coordinate `time`.

In [ ]:
times = pd.date_range('2018-01-01', periods=5)
times

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq='D')

We'll also create arrays to represent sample longitude and latitude:

In [ ]:
lons = np.linspace(-120, -60, 4)
lats = np.linspace(25, 55, 3)

#### Initialize the `DataArray` with complete coordinate info

When we create the `DataArray` instance, we pass in the arrays we just created:

In [ ]:
temp = xr.DataArray(data, coords=[times, lats, lons], dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[276.91512138, 279.86387969, 278.36458624, 273.0550014 ],
        [288.20363005, 277.39943919, 276.14656492, 273.5040345 ],
        [277.05082755, 292.50133624, 283.92811403, 278.48443179]],

       [[283.58293294, 276.11952674, 288.31042283, 281.07499717],
        [285.37992311, 286.49367673, 280.24596734, 285.20312622],
        [287.23002777, 284.54913327, 290.31045538, 278.10829299]],

       [[276.47179621, 284.62000068, 279.92057533, 279.35633454],
        [286.23998898, 280.63052782, 287.89197253, 283.0267185 ],
        [287.91121394, 275.14375092, 292.06369705, 282.1072505 ]],

       [[280.91171542, 280.06454881, 294.56050896, 279.73469873],
        [280.71513053, 283.78874772, 280.85957937, 276.17142697],
        [273.80101824, 287.92413243, 291.89451817, 289.16031958]],

       [[283.49367266, 284.20561716, 284.96659708, 284.60470716],
        [280.27024343, 288.30916921, 279.4805367 , 285.53285659],
        [285.91854135, 277.46997501, 276.42230361, 282.76974697]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

#### Set useful attributes

...and while we're at it, we can also set some attribute metadata:

In [ ]:
temp.attrs['units'] = 'kelvin'
temp.attrs['standard_name'] = 'air_temperature'

temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[276.91512138, 279.86387969, 278.36458624, 273.0550014 ],
        [288.20363005, 277.39943919, 276.14656492, 273.5040345 ],
        [277.05082755, 292.50133624, 283.92811403, 278.48443179]],

       [[283.58293294, 276.11952674, 288.31042283, 281.07499717],
        [285.37992311, 286.49367673, 280.24596734, 285.20312622],
        [287.23002777, 284.54913327, 290.31045538, 278.10829299]],

       [[276.47179621, 284.62000068, 279.92057533, 279.35633454],
        [286.23998898, 280.63052782, 287.89197253, 283.0267185 ],
        [287.91121394, 275.14375092, 292.06369705, 282.1072505 ]],

       [[280.91171542, 280.06454881, 294.56050896, 279.73469873],
        [280.71513053, 283.78874772, 280.85957937, 276.17142697],
        [273.80101824, 287.92413243, 291.89451817, 289.16031958]],

       [[283.49367266, 284.20561716, 284.96659708, 284.60470716],
        [280.27024343, 288.30916921, 279.4805367 , 285.53285659],
        [285.91854135, 277.46997501, 276.42230361, 282.76974697]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

#### Attributes are not preserved by default!

Notice what happens if we perform a mathematical operaton with the `DataArray`: the coordinate values persist, but the attributes are lost. This is done because it is very challenging to know if the attribute metadata is still correct or appropriate after arbitrary arithmetic operations.

To illustrate this, we'll do a simple unit conversion from Kelvin to Celsius:

In [ ]:
temp_in_celsius = temp - 273.15
temp_in_celsius

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[ 3.76512138,  6.71387969,  5.21458624, -0.0949986 ],
        [15.05363005,  4.24943919,  2.99656492,  0.3540345 ],
        [ 3.90082755, 19.35133624, 10.77811403,  5.33443179]],

       [[10.43293294,  2.96952674, 15.16042283,  7.92499717],
        [12.22992311, 13.34367673,  7.09596734, 12.05312622],
        [14.08002777, 11.39913327, 17.16045538,  4.95829299]],

       [[ 3.32179621, 11.47000068,  6.77057533,  6.20633454],
        [13.08998898,  7.48052782, 14.74197253,  9.8767185 ],
        [14.76121394,  1.99375092, 18.91369705,  8.9572505 ]],

       [[ 7.76171542,  6.91454881, 21.41050896,  6.58469873],
        [ 7.56513053, 10.63874772,  7.70957937,  3.02142697],
        [ 0.65101824, 14.77413243, 18.74451817, 16.01031958]],

       [[10.34367266, 11.05561716, 11.81659708, 11.45470716],
        [ 7.12024343, 15.15916921,  6.3305367 , 12.38285659],
        [12.76854135,  4.31997501,  3.27230361,  9.61974697]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

For an in-depth discussion of how Xarray handles metadata, start in the Xarray docs [here](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#approach-to-metadata).

### The `Dataset`: a container for `DataArray`s with shared coordinates

Along with `DataArray`, the other key object type in Xarray is the `Dataset`: a dictionary-like container that holds one or more `DataArray`s, which can also optionally share coordinates (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset)).

The most common way to create a `Dataset` object is to load data from a file (see [below](#Opening-netCDF-data)). Here, instead, we will create another `DataArray` and combine it with our `temp` data.

This will illustrate how the information about common coordinate axes is used.

#### Create a pressure `DataArray` using the same coordinates

This code mirrors how we created the `temp` object above.

In [ ]:
pressure_data = 1000.0 + 5 * np.random.randn(5, 3, 4)
pressure = xr.DataArray(
    pressure_data, coords=[times, lats, lons], dims=['time', 'lat', 'lon']
)
pressure.attrs['units'] = 'hPa'
pressure.attrs['standard_name'] = 'air_pressure'

pressure

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[1001.01825322, 1003.28417269, 1002.35781001,  990.93327136],
        [ 998.55898276, 1001.55169186, 1007.09221515,  995.6741251 ],
        [1005.04082368, 1006.09472263, 1004.78179003,  994.13427345]],

       [[1004.76452165, 1002.90753752,  991.3157723 ,  994.85121594],
        [ 994.302958  ,  995.08282984, 1001.47337817, 1002.84154766],
        [ 999.93806937, 1001.32564333,  995.90515476, 1000.03773792]],

       [[1001.27526936,  996.14287579,  999.54549277, 1005.6854812 ],
        [ 995.15415733, 1003.0217009 , 1006.25185809, 1002.43946596],
        [1004.17535306,  994.44785862,  997.49687825,  993.22439949]],

       [[ 999.44261984,  986.92268081,  996.3470999 ,  991.90533801],
        [1000.60331456, 1002.38917726, 1007.74320976,  994.80957503],
        [1008.89662262,  995.60362321, 1001.04091272,  999.78370795]],

       [[1007.35629116,  996.7772311 ,  996.22709787, 1002.02146263],
        [ 994.63808269,  998.59925128,  995.70473452, 1005.36955056],
        [1008.01125354, 1002.93683014,  998.64473727,  994.01845241]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

#### Create a `Dataset` object

Each `DataArray` in our `Dataset` needs a name! 

The most straightforward way to create a `Dataset` with our `temp` and `pressure` arrays is to pass a dictionary using the keyword argument `data_vars`:

In [ ]:
ds = xr.Dataset(data_vars={'Temperature': temp, 'Pressure': pressure})
ds

<xarray.Dataset>
Dimensions:      (time: 5, lat: 3, lon: 4)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat          (lat) float64 25.0 40.0 55.0
  * lon          (lon) float64 -120.0 -100.0 -80.0 -60.0
Data variables:
    Temperature  (time, lat, lon) float64 276.9 279.9 278.4 ... 276.4 282.8
    Pressure     (time, lat, lon) float64 1.001e+03 1.003e+03 ... 998.6 994.0

Notice that the `Dataset` object `ds` is aware that both data arrays sit on the same coordinate axes.

#### Access Data variables and Coordinates in a `Dataset`

We can pull out any of the individual `DataArray` objects in a few different ways.

Using the "dot" notation:

In [ ]:
ds.Pressure

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[1001.01825322, 1003.28417269, 1002.35781001,  990.93327136],
        [ 998.55898276, 1001.55169186, 1007.09221515,  995.6741251 ],
        [1005.04082368, 1006.09472263, 1004.78179003,  994.13427345]],

       [[1004.76452165, 1002.90753752,  991.3157723 ,  994.85121594],
        [ 994.302958  ,  995.08282984, 1001.47337817, 1002.84154766],
        [ 999.93806937, 1001.32564333,  995.90515476, 1000.03773792]],

       [[1001.27526936,  996.14287579,  999.54549277, 1005.6854812 ],
        [ 995.15415733, 1003.0217009 , 1006.25185809, 1002.43946596],
        [1004.17535306,  994.44785862,  997.49687825,  993.22439949]],

       [[ 999.44261984,  986.92268081,  996.3470999 ,  991.90533801],
        [1000.60331456, 1002.38917726, 1007.74320976,  994.80957503],
        [1008.89662262,  995.60362321, 1001.04091272,  999.78370795]],

       [[1007.35629116,  996.7772311 ,  996.22709787, 1002.02146263],
        [ 994.63808269,  998.59925128,  995.70473452, 1005.36955056],
        [1008.01125354, 1002.93683014,  998.64473727,  994.01845241]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

... or using dictionary access like this:

In [ ]:
ds['Pressure']

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[1001.01825322, 1003.28417269, 1002.35781001,  990.93327136],
        [ 998.55898276, 1001.55169186, 1007.09221515,  995.6741251 ],
        [1005.04082368, 1006.09472263, 1004.78179003,  994.13427345]],

       [[1004.76452165, 1002.90753752,  991.3157723 ,  994.85121594],
        [ 994.302958  ,  995.08282984, 1001.47337817, 1002.84154766],
        [ 999.93806937, 1001.32564333,  995.90515476, 1000.03773792]],

       [[1001.27526936,  996.14287579,  999.54549277, 1005.6854812 ],
        [ 995.15415733, 1003.0217009 , 1006.25185809, 1002.43946596],
        [1004.17535306,  994.44785862,  997.49687825,  993.22439949]],

       [[ 999.44261984,  986.92268081,  996.3470999 ,  991.90533801],
        [1000.60331456, 1002.38917726, 1007.74320976,  994.80957503],
        [1008.89662262,  995.60362321, 1001.04091272,  999.78370795]],

       [[1007.35629116,  996.7772311 ,  996.22709787, 1002.02146263],
        [ 994.63808269,  998.59925128,  995.70473452, 1005.36955056],
        [1008.01125354, 1002.93683014,  998.64473727,  994.01845241]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

We'll return to the `Dataset` object when we start loading data from files.